In [1]:
##############################
### LIBRERÍAS NECESARIAS ####
##############################

import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact
from sklearn import neighbors
import joblib

In [14]:
##############################
### 0. CONEXIÓN A BASE DE DATOS ###
##############################

con = sql.connect('data/db_movies_c1') #Conectarse a la base de datos existente y transportar datos
cur = con.cursor() #Otra conexión (cursor) para ejecutar las consultas en la bd sin traer ni llevar info

# Verifica tablas
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())  # Deberías ver 'full_ratings' en la salida


[('ratings',), ('movies',), ('movies_clean',)]


In [12]:
#############################################
### 1. CONSULTAR LA TABLA full_ratings ######
#############################################

df_peliculas = pd.read_sql("SELECT * FROM full_ratings", conn)
df_peliculas.info()

DatabaseError: Execution failed on sql 'SELECT * FROM full_ratings': no such table: full_ratings

In [ ]:
#######################################################
### 2. PREPARACIÓN DE DATOS PARA MODELOS DE CONTENIDO
#######################################################

# Escalar el año
sc = MinMaxScaler()
df_peliculas["year_scaled"] = sc.fit_transform(df_peliculas[["year"]])

# Seleccionar columnas relevantes para el modelo
columnas_usadas = list(df_peliculas.columns[3:-2]) + ['year_scaled']  # géneros + year
df_dum2 = df_peliculas[columnas_usadas]

# Guardar para uso posterior
joblib.dump(df_dum2, "salidas\\movies_dum2.joblib")

In [ ]:
#####################################################################
### 3. RECOMENDACIÓN MANUAL BASADO EN CONTENIDO (CORRELACIÓN)
#####################################################################

def recomendacion_pelicula(movie=list(df_peliculas['title'])):
    ind_movie = df_peliculas[df_peliculas['title'] == movie].index[0]
    similar_movies = df_dum2.corrwith(df_dum2.iloc[ind_movie, :], axis=1)
    similar_movies = similar_movies.sort_values(ascending=False)
    top_similar_movies = similar_movies.to_frame(name="correlación").iloc[1:11, :]  # omite la misma película
    top_similar_movies['title'] = df_peliculas['title']
    return top_similar_movies

print(interact(recomendacion_pelicula))

In [ ]:
#####################################################################
### 4. RECOMENDACIÓN KNN BASADO EN CONTENIDO (UNA PELÍCULA VISTA)
#####################################################################

# Entrenar modelo KNN
model = neighbors.NearestNeighbors(n_neighbors=11, metric='euclidean')
model.fit(df_dum2)
dist, idlist = model.kneighbors(df_dum2)

# Función interactiva de recomendación
def MovieRecommender(movie_name=list(df_peliculas['title'].value_counts().index)):
    movie_id = df_peliculas[df_peliculas['title'] == movie_name].index[0]
    movie_list_name = []
    for newid in idlist[movie_id]:
        movie_list_name.append(df_peliculas.loc[newid].title)
    return movie_list_name[1:]  # omitir la misma película

print(interact(MovieRecommender))